# Imports

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

# Modified VGG19 for Neural Style Transfer 

In [9]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG, self).__init__()

        self.chosen_features = ["0", "5", "10", "19", "28"]
        self.model = models.vgg19(pretrained=True).features[:29]

    def forward(self, x):
        # Store relevant features
        features = []
        for layer_num, layer in enumerate(self.model):
            x = layer(x)

            if str(layer_num) in self.chosen_features:
                features.append(x)

        return features


In [10]:
def load_image(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imsize = 356

In [12]:
loader = transforms.Compose(
    [
        transforms.Resize((imsize, imsize)),
        transforms.ToTensor(),
    ]
)

In [13]:
original_img = load_image("/content/annahathaway.png")
style_img = load_image("/content/style.jpg")

In [14]:
generated = original_img.clone().requires_grad_(True)
model = VGG().to(device).eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:01<00:00, 298MB/s]


In [15]:
total_steps = 6000
learning_rate = 0.001
alpha = 1
beta = 0.01
optimizer = optim.Adam([generated], lr=learning_rate)

In [16]:
for step in range(total_steps):

    generated_features = model(generated)
    original_img_features = model(original_img)
    style_features = model(style_img)

    style_loss = 0
    original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(
        generated_features, original_img_features, style_features
    ):

        batch_size, channel, height, width = gen_feature.shape
        original_loss += torch.mean((gen_feature - orig_feature) ** 2)
        G = gen_feature.view(channel, height * width).mm(
            gen_feature.view(channel, height * width).t()
        )
        A = style_feature.view(channel, height * width).mm(
            style_feature.view(channel, height * width).t()
        )
        style_loss += torch.mean((G - A) ** 2)

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(total_loss)
        save_image(generated, "generated.png")

tensor(1579634.6250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(84241.6094, device='cuda:0', grad_fn=<AddBackward0>)
tensor(39178.7891, device='cuda:0', grad_fn=<AddBackward0>)
tensor(28035.4316, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22958.8223, device='cuda:0', grad_fn=<AddBackward0>)
tensor(19641.7637, device='cuda:0', grad_fn=<AddBackward0>)
tensor(17160.8711, device='cuda:0', grad_fn=<AddBackward0>)
tensor(15153.3203, device='cuda:0', grad_fn=<AddBackward0>)
tensor(13450.8691, device='cuda:0', grad_fn=<AddBackward0>)
tensor(11972.6250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(10683.7480, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9567.0312, device='cuda:0', grad_fn=<AddBackward0>)
tensor(8606.8037, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7783.1680, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7080.0557, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6476.3057, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5955.1548, device='cuda:0', grad_fn=